# Install libraries (run once locally or always on Colab)

In [13]:
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [1]:
!pip install transformers

In [2]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 13.0 MB/s eta 0:00:00


In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00


# Download Remote Repository

The repo is private. You need to upload it on google drive locally and then cd into it... otherwise just run this notebook locally

In [10]:
# !git clone https://github.com/Ighina/LLMClimate2024.git --> the repo is private!

Cloning into 'LLMClimate2024'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
# After having uploaded the LLMClimate2024 repo on your drive

In [ ]:
cd LLMClimate2024/exp/ClimaBench

In [1]:
# @title LLM Specifications
model_name = "mistralai/Mistral-7B-Instruct-v0.2" # @param ["mistralai/Mistral-7B-Instruct-v0.2"]

## Functions for Inference and Model Instantiation

In [2]:
from typing import List, Dict, Callable, Set
from transformers import AutoModelForCausalLM, AutoTokenizer

def _load_model(model_name: str, bit4: bool = True) -> Callable:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=bit4)
    model.eval()

    return model, tokenizer

def summarise(
    chunk: Dict[str, any],
    model: Callable,
    tokenizer: Callable,
    device: str,
    prompt: str,
) -> str:
    prompt += chunk

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def answer_question(
    question: str,
    prompt: str,
    device: str="cpu"
    ) -> str:
  """
  code mostly from https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

  Parameters
  ----------
  question: str
            the text to answer (text row from the test dataset)

  prompt:   str
            the prompt, i.e. how you ask the LLM to answer. LLMs are
            extremely sensitive to how they are asked to solve a task
            and different LLMs answer differently to the same prompt.

  device:   str
            the device to use: "cpu", for CPU, or "cuda", for GPU. The
            smallest LLMs (7B) need at least 16GB of GPU (i.e. a single
            A100 or 2 T4). If using Google Colab with a single T4, then,
            use CPU (which it should take anyway around 6 second per question).

  Returns
  --------
  decoded:  str
            the answer as output by the LLM.
  """

  messages = [
      {"role": "user", "content": f"{question} {prompt}"}
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_token=True)[0]

  return decoded

# model instantiation
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Load Dataset

In [3]:
# @title Dataset
dataset = "climate_stance" # @param ["climate_stance"]
subset = "validation" # @param ["validation", "test", "train"]

In [4]:
import os

import pandas as pd

from utils import get_climabench_subset

data = get_climabench_subset(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
data[subset]

Dataset({
    features: ['text', 'label', 'idx'],
    num_rows: 354
})

# Define your prompt
This part is the most important when dealing with LLMs. Very small changes can lead to massive differences and, in general, there is no way of doing this but to know model specific tricks and try and error (i.e. prompt engineering).

Check out https://www.kaggle.com/code/lonnieqin/getting-started-with-mistral-7b-instruct for some examples of prompts for Mistral

In [6]:
if dataset=="climate_stance":
  prompt = "With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list"


# Evaluate

In [8]:
import re

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

current_label = []
current_prediction = []
erroneous_answers = []
errors=0
for row in data[subset]:
  text, label, idx = row.values()
  answer = answer_question(text, prompt)
  try:
    # capture the python list that the LLM should output
    # which should include the integer corresponding to the class
    # as defined in the prompt
    filtered_answer=re.findall("\[\d\]", answer[len(prompt)+len(text):])[0]
    prediction = eval(filtered_answer)
    current_label.append(label)
    current_prediction.append(prediction[0])
  except:
    print("this did not work!")
    erroneous_answers.append(answer) # analyse where it fails...
    errors+=1

  if idx%10==0:
    # check the progress
    print(answer)
    print(f1_score(current_label, current_prediction, average="macro"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] well some of them HAVE teeth
Its the #mindlessNess that worries me.
No Science
No Free Press
No Climate Change 
No #Constitutional Law
NO #FACTWitnesses ALLOWED To #Testify
No Defense
Just more from the 
#REPugnantNRABLOODMoneyPARTYWHORES
who didnt BOTHER 2 VET #dumBtyCons #TAXES https://t.co/eoLW0ZP20G With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list [/INST] [0]</s>
1.0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

<s> [INST] Why do we need Sustainable Cities? To support our Global Green Development Goals visit: https://t.co/M7hAwqesjG  #sustainable #cities #sustainability  #globalhealth #Green #development #globaldevelopment #greenbuilding #climatechange #cleanenergy @WorldGBC @USGBC @BREEAMUSA #SDGs https://t.co/xd6S7ALUVO With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list [/INST] [1]</s>
0.3968253968253968


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

<s> [INST] Australia climate change: Thousands rally in Sydney amid bushfires https://t.co/tJnFxBbdsl With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list [/INST] [1]</s>
0.3812187812187812


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

<s> [INST] "Ms. Hillary initially took these treks for the thrill of it, and to enjoy the beauty of the landscape, but she came to understand that climate change was wreaking havoc on the planet, not least its polar extremities, and began lecturing on the topic." https://t.co/PdXFluQynC With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list [/INST] [1]</s>
0.3782935853207339


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
results ={"f1":f1_score(current_label, current_prediction, average="macro"),
          "precision": precision_score(current_label, current_prediction, average="macro"),
          "recall": recall_score(current_label, current_prediction, average="macro"),
          "accuracy": accuracy_score(current_label, current_prediction)}

In [17]:
print(results)

{'f1': 0.3772849314060834, 'precision': 0.3651596941070625, 'recall': 0.4199018295403838, 'accuracy': 0.7133550488599348}


In [18]:
import json

save_results = False
if save_results:
  with open(f"{model_name}-{dataset}-{subset}_results.json") as f:
    json.dump(results, f)

# Analyse mistakes (where the model did not output the correct format)

In [20]:
print(errors) # number of mistakes

47


In [28]:
print(erroneous_answers[5])

<s> [INST] @sneaky_beliky @tms5869 @AntonPagi @jonfitztv @CNN Yep he's just yet another of those who view everything through their hard-core political lens. So the science on climate change must all be garbage because the natural outcome of it is likely to be harmful to the hard-core political stance. Cognitive dissonance is required. With respect to combatting climate change the previous text is: 0: Against, 1: In Favour, 2: Neutral? Output only a python list [/INST] [1, 0, 1, 1, 0]</s>
